In [1]:
#!pip install tqdm
import torchvision.models as models
import sys
import torch
from torch import nn
import time
from tqdm import tqdm
import torch.optim as optim
import copy

# Specify where to find the data preparation class
sys.path.append('../../Data_Preparation')
from Preparation import CustomDataLoader

In [2]:
# VGG16 training data (ImageNet) properties
MEAN = [0.485, 0.456, 0.406]
STD = [0.229, 0.224, 0.225]
DIMENSIONS = 3
#SIZE = 256

In [3]:
# Instantiate the CustomDataLoader class for training
train_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="train", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader
test_data_loader = CustomDataLoader(data_path="../../FER2013_Data", batch_size=16, dataset_type="test", mean=MEAN, std=STD, dimensions=DIMENSIONS).data_loader

# Confirm correct data load
print("Train Data Loader:")
for batch_idx, (inputs, labels) in enumerate(train_data_loader):
    print("Batch Index:", batch_idx)
    print("Inputs Shape:", inputs.shape)
    print("Labels Shape:", labels.shape)
    # Print the first few labels in the batch
    print("Labels:", labels[:5])
    # Break after printing a few batches
    if batch_idx == 2:
        break

Train Data Loader:
Batch Index: 0
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([5, 4, 3, 5, 4])
Batch Index: 1
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([3, 2, 4, 3, 5])
Batch Index: 2
Inputs Shape: torch.Size([16, 3, 299, 299])
Labels Shape: torch.Size([16])
Labels: tensor([4, 4, 2, 2, 0])


In [4]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
vgg19 = models.vgg19(pretrained=True)


m:\My_Private_Files\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
m:\My_Private_Files\Facial_Emotion_Detec-main\.env\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to C:\Users\qo2707es/.cache\torch\hub\checkpoints\vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:08<00:00, 66.0MB/s] 


In [5]:
for param in vgg19.parameters():
    param.requires_grad = False
dataloaders = {'train':train_data_loader, 'val':test_data_loader}


In [6]:
num_classes = len(train_data_loader.dataset.classes)
vgg19.classifier[6] = nn.Linear(4096, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer_vgg19 = optim.Adam(vgg19.parameters())
num_epochs = 15
for epoch in tqdm(range(num_epochs)):
    print('Epoch {}/{}'.format(epoch,num_epochs -1))
    print('-'*10)
    total_batch = len(train_data_loader.dataset) // 16
    for phase in ['train', 'val']:
        if phase == 'train':
            vgg19.train()
        else:
            vgg19.eval()

        running_loss = 0.0
        corrects = 0

        for i,(inputs, labels) in enumerate(dataloaders[phase]):
            optimizer_vgg19.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs_vgg19 = vgg19(inputs)
                _, preds_vgg19 = torch.max(outputs_vgg19, 1)

                loss_vgg19 = criterion(outputs_vgg19, labels)

                if phase == 'train':
                    loss_vgg19.backward()
                    optimizer_vgg19.step()

                    if (i+1) % 5 == 0:
                        print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                             %(epoch+1, num_epochs, i+1, total_batch, loss_vgg19.item()))

            running_loss += loss_vgg19.item() * inputs.size(0)
            corrects += torch.sum(preds_vgg19 == labels.data)

        epoch_loss = running_loss / len(dataloaders[phase])
        epoch_acc = corrects.double() / len(dataloaders[phase])

        print('{} Loss: {:.4f} | Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

  0%|          | 0/15 [00:00<?, ?it/s]

Epoch 0/14
----------
Epoch [1/15], lter [5/1752] Loss: 1.8378
Epoch [1/15], lter [10/1752] Loss: 2.0120
Epoch [1/15], lter [15/1752] Loss: 1.8571
Epoch [1/15], lter [20/1752] Loss: 2.1278
Epoch [1/15], lter [25/1752] Loss: 1.9866
Epoch [1/15], lter [30/1752] Loss: 1.9208
Epoch [1/15], lter [35/1752] Loss: 1.9599
Epoch [1/15], lter [40/1752] Loss: 1.9049
Epoch [1/15], lter [45/1752] Loss: 1.8699
Epoch [1/15], lter [50/1752] Loss: 1.3437
Epoch [1/15], lter [55/1752] Loss: 1.6135
Epoch [1/15], lter [60/1752] Loss: 1.5737
Epoch [1/15], lter [65/1752] Loss: 1.6465
Epoch [1/15], lter [70/1752] Loss: 1.6287
Epoch [1/15], lter [75/1752] Loss: 1.9555
Epoch [1/15], lter [80/1752] Loss: 1.5582
Epoch [1/15], lter [85/1752] Loss: 1.6143
Epoch [1/15], lter [90/1752] Loss: 1.3913
Epoch [1/15], lter [95/1752] Loss: 1.8689
Epoch [1/15], lter [100/1752] Loss: 1.7919
Epoch [1/15], lter [105/1752] Loss: 1.8019
Epoch [1/15], lter [110/1752] Loss: 1.7825
Epoch [1/15], lter [115/1752] Loss: 1.6997
Epoch [1/

  7%|▋         | 1/15 [1:37:16<22:41:44, 5836.06s/it]

val Loss: 25.1029 | Acc: 6.2249
Epoch 1/14
----------
Epoch [2/15], lter [5/1752] Loss: 1.8028
Epoch [2/15], lter [10/1752] Loss: 1.9004
Epoch [2/15], lter [15/1752] Loss: 2.1317
Epoch [2/15], lter [20/1752] Loss: 1.7822
Epoch [2/15], lter [25/1752] Loss: 1.7010
Epoch [2/15], lter [30/1752] Loss: 1.9937
Epoch [2/15], lter [35/1752] Loss: 1.8458
Epoch [2/15], lter [40/1752] Loss: 1.2458
Epoch [2/15], lter [45/1752] Loss: 1.6990
Epoch [2/15], lter [50/1752] Loss: 1.5083
Epoch [2/15], lter [55/1752] Loss: 1.6555
Epoch [2/15], lter [60/1752] Loss: 1.9575
Epoch [2/15], lter [65/1752] Loss: 1.7680
Epoch [2/15], lter [70/1752] Loss: 2.2505
Epoch [2/15], lter [75/1752] Loss: 1.7726
Epoch [2/15], lter [80/1752] Loss: 1.8008
Epoch [2/15], lter [85/1752] Loss: 1.4236
Epoch [2/15], lter [90/1752] Loss: 1.6356
Epoch [2/15], lter [95/1752] Loss: 1.6306
Epoch [2/15], lter [100/1752] Loss: 1.7906
Epoch [2/15], lter [105/1752] Loss: 1.4409
Epoch [2/15], lter [110/1752] Loss: 1.3599
Epoch [2/15], lter [

 13%|█▎        | 2/15 [3:13:55<20:59:46, 5814.37s/it]

...

val Loss: 25.6913 | Acc: 6.3118
Epoch 13/14
----------
Epoch [14/15], lter [5/1752] Loss: 2.2583
Epoch [14/15], lter [10/1752] Loss: 2.1208
Epoch [14/15], lter [15/1752] Loss: 1.6261
Epoch [14/15], lter [20/1752] Loss: 1.8494
Epoch [14/15], lter [25/1752] Loss: 1.5680
Epoch [14/15], lter [30/1752] Loss: 1.7752
Epoch [14/15], lter [35/1752] Loss: 1.3822
Epoch [14/15], lter [40/1752] Loss: 2.5110
Epoch [14/15], lter [45/1752] Loss: 1.9395
Epoch [14/15], lter [50/1752] Loss: 1.6607
Epoch [14/15], lter [55/1752] Loss: 1.3516
Epoch [14/15], lter [60/1752] Loss: 1.1969
Epoch [14/15], lter [65/1752] Loss: 1.9957
Epoch [14/15], lter [70/1752] Loss: 1.9392
Epoch [14/15], lter [75/1752] Loss: 1.8066
Epoch [14/15], lter [80/1752] Loss: 1.8284
Epoch [14/15], lter [85/1752] Loss: 1.6561
Epoch [14/15], lter [90/1752] Loss: 1.8544
Epoch [14/15], lter [95/1752] Loss: 1.7197
Epoch [14/15], lter [100/1752] Loss: 1.6671
Epoch [14/15], lter [105/1752] Loss: 1.5585
Epoch [14/15], lter [110/1752] Loss: 1.47

 93%|█████████▎| 14/15 [22:33:11<1:36:28, 5788.40s/it]

val Loss: 25.7960 | Acc: 6.1871
Epoch 14/14
----------
Epoch [15/15], lter [5/1752] Loss: 2.1542
Epoch [15/15], lter [10/1752] Loss: 1.8546
Epoch [15/15], lter [15/1752] Loss: 2.0107
Epoch [15/15], lter [20/1752] Loss: 1.5865
Epoch [15/15], lter [25/1752] Loss: 2.2277
Epoch [15/15], lter [30/1752] Loss: 2.0538
Epoch [15/15], lter [35/1752] Loss: 2.0384
Epoch [15/15], lter [40/1752] Loss: 1.3813
Epoch [15/15], lter [45/1752] Loss: 1.7163
Epoch [15/15], lter [50/1752] Loss: 1.3390
Epoch [15/15], lter [55/1752] Loss: 1.8286
Epoch [15/15], lter [60/1752] Loss: 1.8720
Epoch [15/15], lter [65/1752] Loss: 1.6088
Epoch [15/15], lter [70/1752] Loss: 2.3396
Epoch [15/15], lter [75/1752] Loss: 1.8490
Epoch [15/15], lter [80/1752] Loss: 1.2760
Epoch [15/15], lter [85/1752] Loss: 1.6023
Epoch [15/15], lter [90/1752] Loss: 1.5686
Epoch [15/15], lter [95/1752] Loss: 1.6648
Epoch [15/15], lter [100/1752] Loss: 1.6427
Epoch [15/15], lter [105/1752] Loss: 1.9725
Epoch [15/15], lter [110/1752] Loss: 2.14

100%|██████████| 15/15 [24:10:27<00:00, 5801.82s/it]  

val Loss: 25.1857 | Acc: 6.3073


In [11]:
# Saving the model
torch.save(vgg19.state_dict(), 'trained_vgg19v15epochs.pt')

RuntimeError: [enforce fail at inline_container.cc:424] . unexpected pos 2373504 vs 2373392

In [12]:
# Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['val']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)

    # Check if the predicted class is equal to the true class
    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 39.57%


In [10]:
#  Initialize the correct_predictions and total_predictions variables
correct_predictions, total_predictions = 0, 0

# Loop over the batches of the test set
for batch_data in dataloaders['train']:
    # Forward pass
    inputs, target = batch_data
    output = vgg19(inputs)

    # Get the predicted class
    _, predicted = torch.max(output.data, 1)

    # Increase the total predictions counter
    total_predictions += target.size(0)


    correct_predictions += (predicted == target).sum().item()

# Calculate the accuracy
accuracy = correct_predictions / total_predictions
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 41.35%
